In [1]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm  # Progress bar
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from urllib.parse import quote
import os

In [2]:
# Load NSE codes from a CSV file
nse_codes_file = 'refined_companies_tick.csv'  # Path to your input CSV file
output_file = 'annual_reports2.csv'  # Path for the final output CSV file
temp_output_file = 'temp_annual_reports2.csv'  # Temporary file to save progress

# Load NSE codes into a list
nse_codes_df = pd.read_csv(nse_codes_file)
nse_codes = nse_codes_df['NSE Code'].tolist()  # Assuming your column is named 'nse_code'

In [3]:
# Load previous results if temp file exists
if os.path.exists(temp_output_file):
    previous_results_df = pd.read_csv(temp_output_file)
    fetched_nse_codes = previous_results_df['nse_code'].unique().tolist()
    print(f"Resuming from checkpoint. {len(fetched_nse_codes)} companies already processed.")
else:
    previous_results_df = pd.DataFrame()
    fetched_nse_codes = []

# Filter out NSE codes that have already been fetched
nse_codes = [code for code in nse_codes if code not in fetched_nse_codes]

# List of rotating User-Agent headers for each request
USER_AGENTS = [
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
]

Resuming from checkpoint. 1834 companies already processed.


In [4]:
def fetch_annual_reports(session, nse_code):
    # Rotate User-Agent for each request
    user_agent = random.choice(USER_AGENTS)

    # URL encode the NSE code to handle special characters like & and spaces
    encoded_nse_code = quote(nse_code)

    # Step 1: Make an initial request to get cookies (no need to encode in this URL)
    session.get(f'https://www.nseindia.com/get-quotes/equity?symbol={nse_code}', headers={
        'User-Agent': user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    })

    # Step 2: Use the session with cookies to fetch the API data (use encoded NSE code)
    url = f"https://www.nseindia.com/api/annual-reports?index=equities&symbol={encoded_nse_code}&issuer={encoded_nse_code}"
    headers = {
        'User-Agent': user_agent,
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': f'https://www.nseindia.com/get-quotes/equity?symbol={encoded_nse_code}',
        'X-Requested-With': 'XMLHttpRequest',
    }

    # Add random delay to slow down requests (longer delay between 5-10 seconds)
    delay = random.uniform(1, 6)
    time.sleep(delay)

    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json().get('data', [])
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred for {nse_code}: {err}")
        return []
    except Exception as err:
        print(f"An error occurred for {nse_code}: {err}")
        return []

In [5]:
# Initialize a requests session with retry and SSL handling
session = requests.Session()
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[429, 500, 502, 503, 504])
adapter = HTTPAdapter(max_retries=retries)
session.mount('https://', adapter)

# Initialize results with previously fetched data if any
results = previous_results_df.to_dict('records') if not previous_results_df.empty else []


In [6]:
# Function to save data incrementally
def save_results():
    results_df = pd.DataFrame(results)
    results_df.to_csv(temp_output_file, index=False)
    print(f"Data saved to {temp_output_file}.")


In [ ]:
# Loop through each NSE code and fetch reports with progress bar
for nse_code in tqdm(nse_codes, desc="Fetching Annual Reports"):
    attempts = 0
    max_attempts = 0  # Set a maximum number of attempts

    while attempts < max_attempts:
        annual_reports = fetch_annual_reports(session, nse_code)
        if annual_reports:
            for report in annual_reports:
                results.append({
                    'nse_code': nse_code,
                    'company_name': report['companyName'],  # Add company name
                    'year': f"{report['fromYr']} - {report['toYr']}",
                    'link': report['fileName']
                })
            # Save progress periodically
            if len(results) % 50 == 0:
                save_results()
            break  # Break if successful
        else:
            attempts += 1
            print(f"Retrying... Attempt {attempts} for {nse_code}")
            time.sleep(1)  # Delay before retrying

Fetching Annual Reports:   0%|                                                                | 0/1929 [00:00<?, ?it/s]

Retrying... Attempt 1 for ABBOTINDIA
Retrying... Attempt 2 for ABBOTINDIA


Fetching Annual Reports:   0%|                                                      | 1/1929 [00:07<4:03:08,  7.57s/it]

Retrying... Attempt 1 for UDAICEMENT
Retrying... Attempt 2 for UDAICEMENT


Fetching Annual Reports:   0%|                                                      | 2/1929 [00:17<4:40:44,  8.74s/it]

...

Fetching Annual Reports:  16%|████████▍                                           | 311/1929 [49:37<4:46:55, 10.64s/it]

Retrying... Attempt 1 for UNITEDINT
Retrying... Attempt 2 for UNITEDINT


Fetching Annual Reports:  16%|████████▍                                           | 312/1929 [49:45<4:27:24,  9.92s/it]

Retrying... Attempt 1 for STRLGUA
Retrying... Attempt 2 for STRLGUA


Fetching Annual Reports:  16%|████████▍                                           | 313/1929 [49:56<4:35:37, 10.23s/it]

In [ ]:
# Save the final results to a CSV file
save_results()  # Final save for all fetched data
results_df = pd.DataFrame(results)
results_df.to_csv(output_file, index=False)
print(f"Annual reports saved to {output_file}.")
